In [17]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd

# 1. 이메일 읽기
with open('received_email.txt', 'r') as file:
    email = file.read().strip()
    print(f"이메일: {email}")

# 2. Firebase Admin SDK 초기화
if not len(firebase_admin._apps):
    cred = credentials.Certificate('C:/pododoc/key/kosmo-96bbe-60906db745e9.json')
    firebase_admin.initialize_app(cred)

# 3. Firestore 클라이언트 생성
db = firestore.client()

# 4. 이메일로 데이터 조회하는 함수 정의
def fetch_data_by_email(email):
    reviews_ref = db.collection('review')
    query = reviews_ref.where('email', '==', email)
    results = query.stream()
    data = []
    for doc in results:
        doc_data = doc.to_dict()
        if 'index' in doc_data and 'rating' in doc_data and 'date' in doc_data and 'photo' in doc_data:
            data.append({
                'index': doc_data['index'],
                'rating': doc_data['rating'],
                'date': doc_data['date'],
                'photo': doc_data['photo']
            })
    return data

# 5. 이메일에 맞는 데이터 가져오기
data = fetch_data_by_email(email)

# 6. 결과 출력
print("가져온 데이터:")
for entry in data:
    print(f"Index: {entry['index']}, Rating: {entry['rating']}, Date: {entry['date']}, Photo: {entry['photo']}")

# 7. Combined_Wine_Data.csv 파일 읽기
wine_data = pd.read_csv('data/Combined_Wine_Data.csv')

# 8. CSV 파일의 컬럼 이름 확인하기
print("CSV 파일의 컬럼 이름:")
print(wine_data.columns)

# 9. index 값에 해당하는 와인 데이터 찾기
indices = [entry['index'] for entry in data]
filtered_wine_data = wine_data[wine_data['index'].isin(indices)]
filtered_wine_data = filtered_wine_data[['index', 'wine_country', 'wine_grape', 'wine_name', 'wine_region', 'wine_winery']]

# 10. Firestore 데이터와 CSV 데이터 결합
firestore_df = pd.DataFrame(data)[['index', 'photo', 'rating']]
combined_data = pd.merge(firestore_df, filtered_wine_data, on='index')

# 11. 결합된 데이터 출력
print("결합된 데이터:")
print(combined_data[['index', 'photo', 'rating', 'wine_country', 'wine_grape', 'wine_name', 'wine_region', 'wine_winery']])


이메일: blue@test.com
가져온 데이터:
Index: 5, Rating: 3.299999952316284, Date: 2024-09-09 11:01:16, Photo: https://firebasestorage.googleapis.com/v0/b/kosmo-96bbe.appspot.com/o/reviewPhoto%2F1725847274521.jpg?alt=media&token=be9dff01-7656-4fae-9eaa-a07184635c85
Index: 2, Rating: 3.0, Date: 2024-09-09 05:47:31, Photo: https://images.vivino.com/thumbs/I0e3d01sQLumhbrSID4aEg_pb_x600.png
Index: 5, Rating: 2.5, Date: 2024-09-09 05:50:44, Photo: https://images.vivino.com/thumbs/eP9Wy_D8SeqfQCTK-fgOzA_pb_x600.png
Index: 734, Rating: 2.0, Date: 2024-09-09 05:48:40, Photo: https://images.vivino.com/thumbs/_IN0qPyiREG_E4qaT8RaMw_pb_x600.png
Index: 180, Rating: 3.5, Date: 2024-09-09 04:24:56, Photo: https://images.vivino.com/thumbs/beMIiKc7Q5uBWNcLl7kXzA_pb_x600.png
Index: 249, Rating: 5.0, Date: 2024-09-09 05:48:09, Photo: https://images.vivino.com/thumbs/fFeVrq8MRG68_d1ha7RoOQ_pb_x600.png
Index: 5, Rating: 2.5999999046325684, Date: 2024-09-09 10:41:29, Photo: https://images.vivino.com/thumbs/eP9Wy_D8Se

c:\Users\KOSMO\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:300: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
